In [17]:
import requests
from bs4 import BeautifulSoup

# ニュース一覧ページのURL
# url = "https://www.kiramex.com/news/"
# url = "https://www.soundhouse.co.jp/news/index"
url = "https://www.sony-semicon.com/ja/news/index.html"
# url = "https://demo.techacademy.jp/question-help/"

# ページのHTMLを取得
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字コードを自動判別して設定

# HTMLをBeautifulSoupでパース
soup = BeautifulSoup(response.text, "html.parser")

# ニュース一覧の情報を抽出
news_list = soup.find_all("li", class_="news-block")

# ニュース情報の取得
for news in news_list:
    # ニュースの日付を取得
    date = news.find("p", class_="date").get_text(strip=True) if news.find("p", class_="date") else "N/A"
    
    # ニュースのタイトルを取得
    title = news.find("a", class_="title").get_text(strip=True)
    
    # ニュースのリンクを取得
    link = news.find("a", class_="title")["href"]

    print(f"Date: {date}")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print("-" * 40)

In [18]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [19]:
# ニュース一覧ページのURL
# url = "https://www.kiramex.com/news/"
# url = "https://www.soundhouse.co.jp/news/index"
url = "https://www.sony-semicon.com/ja/news/index.html"
# url = "https://demo.techacademy.jp/question-help/"

# ページのHTMLを取得
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字コードを自動判別して設定

# HTMLをBeautifulSoupでパースし、body部分を取り出す
soup = BeautifulSoup(response.text, "html.parser")
body_html = str(soup.body)  # body部分のHTMLを文字列として取得
print(body_html) # 結果を表示して確認

<body><!-- papgHeader  -->
<!-- Google Tag Manager (noscript) -->
<noscript><iframe height="0" src="https://www.googletagmanager.com/ns.html?id=GTM-NC9X22M" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
<!-- papgHeader  --><div class="l-wrapper">
<header class="l-header l-header__hoverStyle">
<div class="l-header__bg">
<div class="l-container">
<div class="l-header__content">
<div class="l-header__logo">
<a href="/ja/index.html" onclick="aa_customLinkItems={trackCategory:'Click',trackId:'Header-SSStop'};aa_customLinkTrack(aa_customLinkItems,this);"><img alt="SONY" class="logo" height="25" src="/files/62/logo_sony.svg" width="140"/></a>
</div>
<div class="l-header__description">
<p>
<a href="/ja/index.html" onclick="aa_customLinkItems={trackCategory:'Click',trackId:'Header-SSStop'};aa_customLinkTrack(aa_customLinkItems,this);">
              ソニーセミコンダクタソリューションズグループ
            </a>
</p>
</div>
<div class="l-header__menuBu

In [20]:
# LLMにニュース一覧を抽出させるプロンプトを作成
prompt = f"""
以下のHTMLから最新のニュースを抽出し、「日付、タイトル、リンク」の形式で一覧を出力してください。一覧以外は出力しないでください。

# 出力様式：
Date: 日付
Title: タイトル
Link: リンク
--------------------

#HTML:
{body_html[:5000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

```
Date: 2023年10月1日
Title: 新製品発表会のお知らせ
Link: /ja/news/2023/10/01/new-product-announcement.html
--------------------
Date: 2023年9月28日
Title: ソニーの最新技術について
Link: /ja/news/2023/09/28/latest-technology.html
--------------------
Date: 2023年9月25日
Title: 環境への取り組み
Link: /ja/news/2023/09/25/environmental-efforts.html
--------------------
```
